# Predicting the stock market

## Introduction
In this project, we'll be working with data from the [S&P500 Index](https://en.wikipedia.org/wiki/S%26P_500_Index). The S&P500 is a stock market index. Before we get into what an index is, we'll need to get into the basics of the stock market.

Some companies are publicly traded, which means that anyone can buy and sell their shares on the open market. A share entitles the owner to some control over the direction of the company, and to some percentage (or share) of the earnings of the company. When you buy or sell shares, it's common to say that you're trading a stock.

Stocks are traded daily, and the price can rise or fall from the beginning of a trading day to the end based on demand. Stocks that are in more in demand are traded more often than stocks of smaller companies.

Indexes aggregate the prices of multiple stocks together, and allow you to see how the market as a whole is performing. For example, the Dow Jones Industrial Average aggregates the stock prices of 30 large American companies together. The S&P500 Index aggregates the stock prices of 500 large companies. When an index fund goes up or down, you can say that the underlying market or sector it represents is also going up or down. For example, if the Dow Jones Industrial Average price goes down one day, you can say that American stocks overall went down (ie, most American stocks went down in price).

We'll be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from **1950** to **2015**. We'll **train** the model with data from **1950-2012**, and try to make **predictions** from **2013-2015**.

The dataset is stored in "**sphist.csv**".

## Dataset columns

- **Date**: The date of the record.
- **Open**: The opening price of the day (when trading starts).
- **High**: The highest trade price during the day.
- **Low**: The lowest trade price during the day.
- **Close**: The closing price for the day (when trading is finished).
- **Volume**: The number of shares traded.
- **Adj Close**: The daily closing price, adjusted retroactively to include any corporate actions. 

## Importing packages and reading datasets

In [1]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import itertools

In [2]:
data = pd.read_csv("my_datasets/sphist.csv")
data["Date"] = pd.to_datetime(data["Date"])
data.sort_values(by="Date",inplace=True)

## Exploring the data

It's always interesting to explore some rows of the dataset:

In [3]:
data.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


In [4]:
data.tail()

,Date,Open,High,Low,Close,Volume,Adj Close
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068


## Generating indicators

Stock market data is sequential, and each observation comes a day after the previous observation. Thus, the observations are not all independent, and we can't treat them as such.

This means you have to be extra careful to not inject "future" knowledge into past rows when you do training and prediction. Injecting future knowledge will make our model look good when you're training and testing it, but will make it fail in the real world.

The time series nature of the data means that can generate indicators to make our model more accurate. For instance, we can create a new column that contains the average price of the last 10 trades for each row. This will incorporate information from multiple prior rows into one, and will make predictions much more accurate.

Here are some indicators that are interesting to generate for each row:
- The average price from the past 5 days.
- The average price for the past 30 days.
- The average price for the past 365 days.
- The ratio between the average price for the past 5 days, and the average price for the past 365 days.
- The standard deviation of the price over the past 5 days.
- The standard deviation of the price over the past 365 days.
- The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.

We'll assume that "price" means the "Close" column and days are trading days. 

In [5]:
def average_days_column(df, days):
    new_column = "AVG_" + str(days)
    new_values = df["Close"].rolling(days).mean()
    new_values = new_values.shift(periods=1)
    df[new_column] = new_values
    return df

def deviation_days_column(df, days):
    new_column = "STD_" + str(days)
    new_values = df["Close"].rolling(days).std()
    new_values = new_values.shift(periods=1)
    df[new_column] = new_values
    return df

In [6]:
data = average_days_column(data,5)
data = average_days_column(data,30)
data = average_days_column(data,365)
data["AVG_RATIO"] = data["AVG_5"]/data["AVG_365"]
data = deviation_days_column(data,5)
data = deviation_days_column(data,365)
data["STD_RATIO"] = data["STD_5"]/data["STD_365"]

In [7]:
new_features = ["AVG_5","AVG_30","AVG_365","AVG_RATIO",
                "STD_5","STD_365","STD_RATIO"]

## Splitting up the data

Since we're computing indicators that use historical data, there are some rows where there isn't enough historical data to generate them. Some of the indicators use 365 days of historical data, and the dataset starts on 1950-01-03. Thus, any rows that fall before 1951-01-03 don't have enough historical data to compute all the indicators. We'll need to remove these rows before you split the data.

In [8]:
print("Initial rows:", data.shape[0])

data = data.dropna(axis = 0)

print("Final rows:", data.shape[0])

Initial rows: 16590
Final rows: 16225


In [9]:
train = data[data["Date"] < datetime(year=2013, month=1, day=1)]
test = data[data["Date"] >= datetime(year=2013, month=1, day=1)]

print("Train rows:", train.shape[0])
print("Test rows:", test.shape[0])

Train rows: 15486
Test rows: 739


## Making predictions

We can now define an error metric, train a model using the train data and make predictions on the test data.

We'll use Mean Absolute Error, also called MAE, as an error metric, because it will show you how "close" we were to the price in intuitive terms. Mean Squared Error, or MSE, is an alternative that is more commonly used, but makes it harder to intuitively tell how far off we are from the true price because it squares the error.

In [10]:
def calculate_mae(features_comb,target,train,test):
    results_dict = {}
    for combination in features_comb:
        lr = LinearRegression()
        lr.fit(train[combination],train[target])
        predictions = lr.predict(test[combination])
        mae = mean_absolute_error(predictions,test[target])
        combination_name = ", ".join(c for c in combination)
        results_dict[combination_name] = mae
    return pd.Series(results_dict)

In [11]:
target = "Close"
features_comb = []

#Use combinations module to get all features combination
for L in range(1, len(new_features)+1):
    for subset in itertools.combinations(new_features, L):
        features_comb.append(list(subset))

In [12]:
features_mae = calculate_mae(features_comb,target,train,test)
features_mae = features_mae.sort_values()

print(features_mae.head(10))

AVG_5, AVG_365, AVG_RATIO, STD_5, STD_RATIO    16.090330
AVG_5, AVG_365, AVG_RATIO, STD_5               16.093178
AVG_5, AVG_365, STD_5, STD_RATIO               16.093650
AVG_5, AVG_365, STD_5                          16.094213
AVG_5, AVG_365, AVG_RATIO, STD_RATIO           16.100016
AVG_5, AVG_RATIO, STD_5                        16.113020
AVG_5, AVG_365, STD_RATIO                      16.113100
AVG_5, AVG_RATIO, STD_5, STD_RATIO             16.114180
AVG_5, STD_5                                   16.118299
AVG_5, STD_5, STD_RATIO                        16.118897
dtype: float64


## Conclusion

According to the results, there is no combination with an outstanding accuracy. Instead, the minimum mean absolute error stays at **16**.

Taking a look on the numbers, a good approach could be to take the model with the indicators "AVG_5" and "STD_5". In this case, we get a mae value (16.118299) close to the maximum by just using 2 features.

However, considering that daily changes could be less than 16, we should think in new indicators and try to find lower mae values.